In [1]:
import tensorflow as tf
a = tf.constant([1.0,2.0], name="a")
b = tf.constant([2.0,3.0], name="b")
result = a+b

C:\Users\y63qiu\AppData\Local\Continuum\anaconda3\envs\yorkml\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
print(a.graph is tf.get_default_graph())

True


In [3]:
from IPython.display import clear_output, Image, display, HTML
import numpy as np    

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [4]:
show_graph(tf.get_default_graph().as_graph_def())

In [5]:
# 如何在不同的计算图上定义和使用变量

g1 = tf.Graph()
with g1.as_default():
    v = tf.get_variable("v", shape=[1], initializer=tf.zeros_initializer)
    
g2 = tf.Graph()
with g2.as_default():
    v = tf.get_variable("v", shape=[1], initializer=tf.ones_initializer)
    
with tf.Session(graph=g1) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable("v")))

[0.]


In [6]:
# 用 tf.Graph.device 来指定 GPU 为运算设备
g = tf.Graph()

with g.device('/gpu:0'):
    result = a + b

In [7]:
sess = tf.Session()

with sess.as_default():
    print(result.eval())

[3. 5.]


In [8]:
sess = tf.Session()

print(sess.run(result))

[3. 5.]


In [9]:
print(result.eval(session=sess))

[3. 5.]


# 一个完整的神经网络样例程序（二分类问题） 

In [11]:
import tensorflow as tf
import numpy as np
from numpy.random import RandomState

In [39]:
batch_size = 8

w1 = tf.Variable(tf.random_normal([2,3],stddev=1, seed=42))
w2 = tf.Variable(tf.random_normal([3,1],stddev=1, seed=42))

X_ = tf.placeholder(tf.float32, shape=(None,2), name='x-input')
Y_ = tf.placeholder(tf.float32, shape=(None,1), name='y-input')

a = tf.matmul(X_,w1)
y = tf.matmul(a,w2)

y = tf.sigmoid(y)
cross_entropy = -tf.reduce_mean(Y_ * tf.log(tf.clip_by_value(y, 1e-10, 1.0))
                               + (1-Y_)*tf.log(tf.clip_by_value(1-y, 1e-10, 1.0)))
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

rdm = RandomState(42)
dataset_size = 128
X = rdm.rand(dataset_size,2)

# x1 + x2 大于 1 的被视为正例， 小于 1 的被视为反例
Y = [[int(x1+x2>1)] for x1,x2 in X]

with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    
    sess.run(init_op)
    
    print(sess.run(w1))
    print(sess.run(w2))
    
    steps = 5000
    for i in range(steps):
        start = (i * batch_size) % dataset_size
        end = min(start+batch_size, dataset_size)
        
        sess.run(train_step, feed_dict={X_:X[start:end], Y_:Y[start:end]})
        if i % 1000 == 0:
            total_cross_entropy = sess.run(cross_entropy, feed_dict={X_:X, Y_:Y})
            
            print("After %d training step(s), cross entropy on all data is %g" %(i, total_cross_entropy))
            
    print(sess.run(w1))
    print(sess.run(w2))

[[-0.28077507 -0.13775212 -0.6763296 ]
 [ 0.02458041 -0.89358455 -0.82847327]]
[[-0.28077507]
 [-0.13775212]
 [-0.6763296 ]]
After 0 training step(s), cross entropy on all data is 0.67613
After 1000 training step(s), cross entropy on all data is 0.661944
After 2000 training step(s), cross entropy on all data is 0.657742
After 3000 training step(s), cross entropy on all data is 0.656848
After 4000 training step(s), cross entropy on all data is 0.656715
[[-0.589029    0.12366031 -0.93437386]
 [ 0.48000205 -1.0345082  -0.4361722 ]]
[[-0.86661786]
 [ 0.21232782]
 [-0.68869597]]


# 实现卷积层

In [42]:
filter_weight = tf.get_variable(
    'weights',[5,5,3,16], initializer=tf.truncated_normal_initializer(stddev=0.1))

In [43]:
biases = tf.get_variable(
    'biases',[16], initializer=tf.constant_initializer(0.1))

In [47]:
conv = tf.nn.conv2d(
    input_tensor, filter_weight, strides=[1,1,1,1], padding="SAME")

NameError: name 'input_tensor' is not defined